# Imports

In [1]:
import pynq
import time
from pynq import PL, DefaultIP
import xrfclk
import xrfdc
import numpy as np
from fpbinary import FpBinary
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq, fftshift
import mkidgen3 as g3
import mkidgen3.mkidpynq as mkidpynq
import mkidgen3.daccomb as daccomb
import mkidgen3
import mkidgen3.util
from mkidgen3.daccomb import generate_dac_comb
from mkidgen3.drivers import axiswitch, bintores, capture, ddc, dactable, axififo, rfdc
from mkidgen3.ifboard import IFBoard
import logging
from logging import getLogger
import json

In [2]:
logging.basicConfig()
logging.getLogger('mkidgen3').setLevel('DEBUG')

In [3]:
class SwitchOnLast(DefaultIP):
    bindto = ['mazinlab:mkidgen3:switch_on_last:0.1']

    def __init__(self, description):
        super().__init__(description=description)
    
    def disable(self):
        self.register_map.enable = False
        
    def set_driver(self, slave=0, commit=True):
        """Set the input"""
        slave = max(min(slave, 5), 0)
        #self.disable()
        time.sleep(.1)
        self.register_map.stream = slave
        time.sleep(.1)
        if commit:
            self.register_map.enable=True

    def is_disabled(self, master=0):
        return self.register_map.enable.enable

    def driver_for(self, master=0):
        return self.register_map.stream.stream

    def commit(self):
        """Commit config, triggers a soft 16 cycle reset"""
        self.register_map.enable=True


# Load the Overlay and Mig Allocation XCLBIN

In [4]:
ol = g3.configure('fifoclk/iqtest.bit', ignore_version=True, clocks=True, external_10mhz=True)

In [5]:
print(f"PL Bitfile: {PL.bitfile_name}\nPL Timestamp: {PL.timestamp}\n"
      f"Overlay timestamp: {ol.timestamp}  Loaded: {ol.is_loaded()}")

PL Bitfile: /home/xilinx/jupyter_notebooks/captest/fifoclk/iqtest.bit
PL Timestamp: 2022/3/8 20:54:26 +599988
Overlay timestamp: 2022/3/8 20:54:26 +599988  Loaded: True


#manual

In [6]:
rfdc=ol.rfdc
axis2mm=ol.capture.axis2mm
filter_iq = ol.capture.filter_iq_0
iqgen=ol.iq_gen_0
axiint = ol.axi_intc_0
ol.iq_gen_0.register_map.max=256*(2**19-1)
ol.iq_gen_0.register_map.run=True
ol.iq_gen_0.register_map

INFO:mkidgen3.drivers.rfdc:RFDCHierarchy does not support switching ADCs
DEBUG:mkidgen3.drivers.capture:Capture of ddciq not supported


RegisterMap {
  max = Register(max=1048320, RESERVED=0),
  run = Register(run=1, RESERVED=0)
}

In [ ]:
ol.capture.switch.set_driver(0)

In [7]:
ol.capture.filter_phase.keep='all'
ol.capture.filter_iq['rawiq'].keep='all'

In [ ]:
ol.capture.axis_switch.register_map.enable=True
ol.capture.axis_switch.register_map.stream=3
ol.capture.axis_switch.register_map

In [ ]:
self=ol.capture
tap_location='rawiq'
groups='all'
n=2000
self.filter_iq[tap_location].keep = groups
n_groups = len(self.filter_iq[tap_location].keep)
capture_bytes = n * n_groups * 32
buffer = pynq.allocate((n, n_groups * 8, 2), dtype='i2', target=self.ddr4_0)
addr = buffer.device_address
source=tap_location
n=capture_bytes

if n % 64:
    raise ValueError('Can only capture in multiples of 64 bytes')
self.switch.set_driver(slave=self.SOURCE_MAP[source], commit=False)
if not self.axis2mm.ready:
    raise IOError("capture core not ready, this shouldn't happen."
                  " Try calling .axis2mm.abort() followed by .axis2mm.clear_error()"
                  " then try a small throwaway capture (data order may not be aligned in the first capture "
                  "after a reset).")
getLogger(__name__).debug(f'Starting capture of {n} bytes ({n // 64} beats) to address {hex(addr)} from '
                          f'source {source}')
self.axis2mm.addr = addr
self.axis2mm.len = n
self.axis2mm.start(continuous=False, increment=True)
self.switch.commit()
axis2mm.cmd_ctrl_reg

In [ ]:
axis2mm.cmd_ctrl_reg

Now try phase

In [ ]:
self=ol.capture
tap_location='phase'
groups='all'
n=2000

self.filter_phase.keep = groups
n_groups = len(self.filter_phase.keep)
capture_bytes = n * 2 * n_groups * 16
buffer = pynq.allocate((n, n_groups * 16), dtype='i2', target=self.ddr4_0)

addr = buffer.device_address
source=tap_location
n=capture_bytes

if n % 64:
    raise ValueError('Can only capture in multiples of 64 bytes')
self.switch.set_driver(slave=self.SOURCE_MAP[source], commit=False)
if not self.axis2mm.ready:
    raise IOError("capture core not ready, this shouldn't happen."
                  " Try calling .axis2mm.abort() followed by .axis2mm.clear_error()"
                  " then try a small throwaway capture (data order may not be aligned in the first capture "
                  "after a reset).")
getLogger(__name__).debug(f'Starting capture of {n} bytes ({n // 64} beats) to address {hex(addr)} from '
                          f'source {source}')
self.axis2mm.addr = addr
self.axis2mm.len = n
self.axis2mm.start(continuous=False, increment=True)
self.switch.commit()
self.axis2mm.cmd_ctrl_reg

In [ ]:
self.axis2mm.cmd_ctrl_reg, ol.capture.switch.register_map

In [ ]:
self.filter_iq['rawiq'].keep = 'all'

In [ ]:
ol.capture.switch.register_map.enable=1

## Fetch Blocks

ol.capture.SOURCE_MAP = dict(adc=0, iq=2, rawiq=1, phase=3)

In [ ]:
rfdc=ol.rfdc
axis2mm=ol.capture.axis2mm
filter_iq = ol.capture.filter_iq_0
iqgen=ol.iq_gen_0
axiint = ol.axi_intc_0
ddr4_mmio=pynq.MMIO(g3.mkidpynq.PL_DDR4_ADDR, length=2**32)

Start a ramp running, show axis2mm status

In [ ]:
iqgen.register_map.max=256*(2**19-1)
iqgen.register_map.run=True
iqgen.register_map

In [ ]:
iqgen.register_map

In [ ]:
axis2mm.cmd_ctrl_reg

Example of how to manually start a stream flowing up to the stream limit block

ol.capture.switch.set_driver(slave=ol.capture.SOURCE_MAP['rawiq'], commit=True)

ol.capture.filter_iq['rawiq'].keep='all'
ol.capture.filter_iq['rawiq'].register_map

## Capture data for iq and look at it

iq vals are 32bit uints on a ramp, sloppily grab the real and subtract the first value from each capture.
Resulting values, raveled out of their resonator channels, should count by 1 until they wrap at MAX_UINT16 and then repeat, but you need to be careful that the - value isn't caused by doing np.diff on a uint

In [8]:
def rc():
    time.sleep(np.random.uniform(0,1))
    return ol.capture.capture_iq(2000, groups='all', tap_location='rawiq')
d=[rc() for i in range(8)]

DEBUG:mkidgen3.drivers.capture:Capturing 15.6 MB of data @ 16384.0 MBps. ETA 1 ms
DEBUG:mkidgen3.drivers.capture:Capturing 15.6 MB of data @ 16384.0 MBps. ETA 1 ms
DEBUG:mkidgen3.drivers.capture:Capturing 15.6 MB of data @ 16384.0 MBps. ETA 1 ms
DEBUG:mkidgen3.drivers.capture:Capturing 15.6 MB of data @ 16384.0 MBps. ETA 1 ms
DEBUG:mkidgen3.drivers.capture:Capturing 15.6 MB of data @ 16384.0 MBps. ETA 1 ms
DEBUG:mkidgen3.drivers.capture:Capturing 15.6 MB of data @ 16384.0 MBps. ETA 1 ms
DEBUG:mkidgen3.drivers.capture:Capturing 15.6 MB of data @ 16384.0 MBps. ETA 1 ms
DEBUG:mkidgen3.drivers.capture:Capturing 15.6 MB of data @ 16384.0 MBps. ETA 1 ms


In [9]:
#NB if samples grows past ~2000 this starts to get a bit slow
x=[]
for v in d:
    a=v.astype(np.uint16, copy=False)
    a=np.array(a[...,0].reshape(-1))
    a-=a[0]
    x.append(a)
x=np.array(x)
axis2mm.cmd_ctrl_reg

{'r_busy': False,
 'r_err': False,
 'r_complete': True,
 'r_continuous': False,
 'r_increment_n': False,
 'r_tlast_syncd_n': True,
 'decode_error': False,
 'slave_error': False,
 'overflow_error': False,
 'aborting': False,
 'fifo_len': 8,
 'abort': 0}

If this isn't 1, 0 (or possibly -MAX_UINT16) there is a problem

In [10]:
xd=np.diff(x.astype(np.int32),axis=1)
xd.max(), xd.min()

(1, -65535)

In [11]:
xi=x.astype(np.int32)
for i in list(range(17))+[71,72]+list(range(2047-16,2048)):
    print(f'Beat {i//16:3}, {i//16+128:3}, ...  lane {i%8} chan {i:4}: {xi[0,i::2048][:10]}')

Beat   0, 128, ...  lane 0 chan    0: [    0  2048  4096  6144  8192 10240 12288 14336 16384 18432]
Beat   0, 128, ...  lane 1 chan    1: [    1  2049  4097  6145  8193 10241 12289 14337 16385 18433]
Beat   0, 128, ...  lane 2 chan    2: [    2  2050  4098  6146  8194 10242 12290 14338 16386 18434]
Beat   0, 128, ...  lane 3 chan    3: [    3  2051  4099  6147  8195 10243 12291 14339 16387 18435]
Beat   0, 128, ...  lane 4 chan    4: [    4  2052  4100  6148  8196 10244 12292 14340 16388 18436]
Beat   0, 128, ...  lane 5 chan    5: [    5  2053  4101  6149  8197 10245 12293 14341 16389 18437]
Beat   0, 128, ...  lane 6 chan    6: [    6  2054  4102  6150  8198 10246 12294 14342 16390 18438]
Beat   0, 128, ...  lane 7 chan    7: [    7  2055  4103  6151  8199 10247 12295 14343 16391 18439]
Beat   0, 128, ...  lane 0 chan    8: [    8  2056  4104  6152  8200 10248 12296 14344 16392 18440]
Beat   0, 128, ...  lane 1 chan    9: [    9  2057  4105  6153  8201 10249 12297 14345 16393 18441]


Its a good idea to go through this section a few times with different capture lengths, all should be fine.

## Capture data for phase and look at it, this will trigger a stream switch

phase data is on a slightly different ramp, with each group of 4 phases counting the stream beat. So we expect chan 0-3 to count 0,512,1024,... chan 4-7 to count 1,513,1025,...

In [12]:
def rc():
    time.sleep(np.random.uniform(0,1))
    return ol.capture.capture_phase(200, groups='all')
d=[rc() for i in range(8)]

DEBUG:mkidgen3.drivers.capture:Capturing ~0.78 MB of data @ 4096.0 MBps. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing ~0.78 MB of data @ 4096.0 MBps. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing ~0.78 MB of data @ 4096.0 MBps. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing ~0.78 MB of data @ 4096.0 MBps. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing ~0.78 MB of data @ 4096.0 MBps. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing ~0.78 MB of data @ 4096.0 MBps. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing ~0.78 MB of data @ 4096.0 MBps. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing ~0.78 MB of data @ 4096.0 MBps. ETA 0 ms


In [ ]:
ol.capture.switch.register_map.enable=0
ol.capture.switch.register_map.stream=0

In [ ]:
axis2mm.cmd_ctrl_reg

In [13]:
x=np.array([x.astype(np.int32, copy=False).reshape(-1)-x.flat[0].astype(np.int32) for x in d])

In [14]:
xi=x.astype(np.int32)
for i in list(range(17))+list(range(2047-16,2048)):
    print(f'Beat {i//16:3}, {i//16+128:3}, ...  lane {i%4} chan {i:4}: {xi[0,i::2048][:10]}')

Beat   0, 128, ...  lane 0 chan    0: [     0    512   1024   1536   2048 -62976 -62464 -61952 -61440 -60928]
Beat   0, 128, ...  lane 1 chan    1: [     0    512   1024   1536   2048 -62976 -62464 -61952 -61440 -60928]
Beat   0, 128, ...  lane 2 chan    2: [     0    512   1024   1536   2048 -62976 -62464 -61952 -61440 -60928]
Beat   0, 128, ...  lane 3 chan    3: [     0    512   1024   1536   2048 -62976 -62464 -61952 -61440 -60928]
Beat   0, 128, ...  lane 0 chan    4: [     1    513   1025   1537   2049 -62975 -62463 -61951 -61439 -60927]
Beat   0, 128, ...  lane 1 chan    5: [     1    513   1025   1537   2049 -62975 -62463 -61951 -61439 -60927]
Beat   0, 128, ...  lane 2 chan    6: [     1    513   1025   1537   2049 -62975 -62463 -61951 -61439 -60927]
Beat   0, 128, ...  lane 3 chan    7: [     1    513   1025   1537   2049 -62975 -62463 -61951 -61439 -60927]
Beat   0, 128, ...  lane 0 chan    8: [     2    514   1026   1538   2050 -62974 -62462 -61950 -61438 -60926]
Beat   0, 

All had better be 1

In [15]:
np.diff(x[:,::4].astype(np.int32), axis=1).max(1)

array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

## ADC Capture

In [16]:
def rc():
    time.sleep(np.random.uniform(0,1))
    return ol.capture.capture_adc(256*200)
d=[rc() for i in range(8)]

DEBUG:mkidgen3.drivers.capture:Capturing 0.1953125 MB of data @ 16384 MB/s. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing 0.1953125 MB of data @ 16384 MB/s. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing 0.1953125 MB of data @ 16384 MB/s. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing 0.1953125 MB of data @ 16384 MB/s. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing 0.1953125 MB of data @ 16384 MB/s. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing 0.1953125 MB of data @ 16384 MB/s. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing 0.1953125 MB of data @ 16384 MB/s. ETA 0 ms
DEBUG:mkidgen3.drivers.capture:Capturing 0.1953125 MB of data @ 16384 MB/s. ETA 0 ms


In [17]:
x=np.array([x[:,0].astype(np.uint16, copy=False)-x.flat[0].astype(np.uint16) for x in d])

In [18]:
np.diff(x.astype(np.int32), axis=1).max(1),np.diff(x.astype(np.int32), axis=1).min(1)

(array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int32),
 array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))

In [19]:
xi=x.astype(np.int32)
for i in list(range(17))+list(range(2047-16,2048)):
    print(f'Beat {i//16:3}, {i//16+128:3}, ...  lane {i%8} chan {i:4}: {xi[0,i::2048][:10]}')

Beat   0, 128, ...  lane 0 chan    0: [    0  2048  4096  6144  8192 10240 12288 14336 16384 18432]
Beat   0, 128, ...  lane 1 chan    1: [    1  2049  4097  6145  8193 10241 12289 14337 16385 18433]
Beat   0, 128, ...  lane 2 chan    2: [    2  2050  4098  6146  8194 10242 12290 14338 16386 18434]
Beat   0, 128, ...  lane 3 chan    3: [    3  2051  4099  6147  8195 10243 12291 14339 16387 18435]
Beat   0, 128, ...  lane 4 chan    4: [    4  2052  4100  6148  8196 10244 12292 14340 16388 18436]
Beat   0, 128, ...  lane 5 chan    5: [    5  2053  4101  6149  8197 10245 12293 14341 16389 18437]
Beat   0, 128, ...  lane 6 chan    6: [    6  2054  4102  6150  8198 10246 12294 14342 16390 18438]
Beat   0, 128, ...  lane 7 chan    7: [    7  2055  4103  6151  8199 10247 12295 14343 16391 18439]
Beat   0, 128, ...  lane 0 chan    8: [    8  2056  4104  6152  8200 10248 12296 14344 16392 18440]
Beat   0, 128, ...  lane 1 chan    9: [    9  2057  4105  6153  8201 10249 12297 14345 16393 18441]


In [ ]:
try:
    print(x[x!=x.max()].max())
except ValueError:
    pass